In [ ]:
import datetime
import geopandas
import joblib
import pathlib
import xyzservices

In [ ]:
CACHE_DIRECTORY = pathlib.Path() / ".cache"
CACHE_TIME = datetime.timedelta(weeks=3)

CACHE_DIRECTORY.mkdir(exist_ok=True)
memory = joblib.Memory(CACHE_DIRECTORY, verbose=0)
memory.reduce_size(age_limit=CACHE_TIME)

In [ ]:
WFS_BASE_URL = (
    "https://data.wien.gv.at/daten/geo"
    "?service=WFS"
    "&version=2.0.0"
    "&request=GetFeature"
    "&srsName=EPSG:31256"
    "&typeNames={:s}"
)

FMZK_VERKEHR1_WFS_URL = WFS_BASE_URL.format("ogdwien:FMZKVERKEHR1OGD")
FMZK_VERKEHR2_WFS_URL = WFS_BASE_URL.format("ogdwien:FMZKVERKEHR2OGD")
ORTHOFOTO_TILE_URL = "https://mapsneu.wien.gv.at/wmts/lb/farbe/google3857/{z}/{y}/{x}.jpeg"

In [ ]:
GEHSTEIGE_FILL_COLOUR = "#83ba7e"
GEHSTEIGE_STROKE_COLOUR = "#83ba7e"
PARKPLAETZE_FILL_COLOUR = "#73a0dc"
PARKPLAETZE_STROKE_COLOUR = "#73a0dc"

In [ ]:
@memory.cache
def cached_geopandas_read_file(*args, **kwargs):
    return geopandas.read_file(*args, **kwargs)

In [ ]:
fmzk_verkehr1 = cached_geopandas_read_file(FMZK_VERKEHR1_WFS_URL)
parkplaetze = fmzk_verkehr1[fmzk_verkehr1.LAYER == "Ruhender Verkehr"]
parkplaetze.crs = "EPSG:31256"
del fmzk_verkehr1

parkplaetze

In [ ]:
fmzk_verkehr2 = cached_geopandas_read_file(FMZK_VERKEHR2_WFS_URL)
gehsteige = fmzk_verkehr2[fmzk_verkehr2.LAYER == "Gehsteig"]
gehsteige.crs = "EPSG:31256"
del fmzk_verkehr2

gehsteige

In [ ]:
uebersichtskarte = gehsteige.explore(
    style_kwds={
        "fillColor": GEHSTEIGE_FILL_COLOUR,
        "fillOpacity": 0.9,
        "color": GEHSTEIGE_STROKE_COLOUR,
        "weight": 0.1,
    },
    
    map_kwds={
        "center": {"lat": 48.2515, "lng": 16.4549}
    },
    zoom_start=18,
    tiles=ORTHOFOTO_TILE_URL,
    attr="Datenquelle: Stadt Wien – data.wien.gv.at",
)
uebersichtskarte = parkplaetze.explore(
    m=uebersichtskarte,
    style_kwds={
        "fillColor": PARKPLAETZE_FILL_COLOUR,
        "fillOpacity": 0.9,
        "color": PARKPLAETZE_STROKE_COLOUR,
        "weight": 0.1,
    },
)
uebersichtskarte